In [36]:
import pandas as pd
import sqlite3
import requests

url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = pd.json_normalize(data)
    print("Data loaded successfully!")
else:
    print("Failed to load data")

df['date_utc'] = pd.to_datetime(df['date_utc'], errors='coerce')
df['year'] = df['date_utc'].dt.year
df['success'] = df['success'].fillna(False).astype(int)

columns_to_keep = ['name', 'date_utc', 'year', 'success', 'rocket', 'launchpad', 'details']
df_sql = df[columns_to_keep].copy()

conn = sqlite3.connect("spacex.db")
df_sql.to_sql("launches", conn, if_exists='replace', index=False)
print("Data saved to SQLite database.")

query_1 = "SELECT COUNT(*) AS total_launches FROM launches"
print(pd.read_sql(query_1, conn))

query_2 = """
SELECT year, COUNT(*) AS launches
FROM launches
GROUP BY year
ORDER BY year
"""
print(pd.read_sql(query_2, conn))

query_3 = """
SELECT success, COUNT(*) AS count
FROM launches
GROUP BY success
"""
print(pd.read_sql(query_3, conn))

query_4 = """
SELECT launchpad, COUNT(*) AS count
FROM launches
GROUP BY launchpad
ORDER BY count DESC
"""
print(pd.read_sql(query_4, conn))

query_5 = """
SELECT rocket, COUNT(*) AS count
FROM launches
GROUP BY rocket
ORDER BY count DESC
LIMIT 5
"""
print(pd.read_sql(query_5, conn))

# Optional: Launches with missing details
query_6 = """
SELECT name, date_utc
FROM launches
WHERE details IS NULL
"""
print(pd.read_sql(query_6, conn))

# Close DB connection
conn.close()


✅ Data loaded successfully!
📁 Data saved to SQLite database.
   total_launches
0             205
    year  launches
0   2006         1
1   2007         1
2   2008         2
3   2009         1
4   2010         2
5   2012         2
6   2013         3
7   2014         6
8   2015         7
9   2016         9
10  2017        18
11  2018        21
12  2019        13
13  2020        26
14  2021        31
15  2022        62
   success  count
0        0     24
1        1    181
                  launchpad  count
0  5e9e4501f509094ba4566f84    112
1  5e9e4502f509094188566f88     58
2  5e9e4502f509092b78566f87     30
3  5e9e4502f5090995de566f86      5
                     rocket  count
0  5e9d0d95eda69973a809d1ec    195
1  5e9d0d95eda69974db09d1ed      5
2  5e9d0d95eda69955f709d1eb      5
                      name                   date_utc
0                 RazakSat  2009-07-13 03:35:00+00:00
1     Falcon 9 Test Flight  2010-06-04 18:45:00+00:00
2                   COTS 1  2010-12-08 15:43:00+0

C:\Users\kash7\AppData\Local\Temp\ipykernel_5008\3496229673.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['success'] = df['success'].fillna(False).astype(int)
